# web scraping assignment-3(selenium-exception handling)

In [1]:
#importing all required libraries
import selenium                # library to work with selenium
from selenium import webdriver # library to open up automated chrome window
import pandas as pd            # to make dataframe
import numpy as np             # to make numerical operations
import time                    # to stop search enginge for few seconds
import warnings
warnings.filterwarnings('ignore')  # ignores any kind of warnings
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import requests

=======================================================================================================================

Q1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [223]:
# first connect to the web driver
driver1=webdriver.Chrome('chromedriver.exe')
time.sleep(5)

In [224]:
# opening up amazon.in website on automated webdriver
driver1.get('https://www.amazon.in/')

In [225]:
#finding elements for user input
search_product1=driver1.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')
search_product1.send_keys('branded tie')

In [226]:
# clicking search button
click_search_button1=driver1.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input')
click_search_button1.click()

Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [227]:
#creating empty lists to store details
brand_name1=[]
name_of_product1=[]
price1=[]
product_url1=[]

for i in range(0,3):
    
    #scraping brand name
    brand_tags1=driver1.find_elements_by_xpath('//h5[@class="s-line-clamp-1"]')
    for a in brand_tags1:
        brand_name1.append(a.text)
    
    #scraping name of the product
    name_tags1=driver1.find_elements_by_xpath('//span[@class="a-size-base-plus a-color-base a-text-normal"]')
    for b in name_tags1:
        name_of_product1.append(b.text)
        
    #scraping price
    price_tags1=driver1.find_elements_by_xpath('//span[@class="a-price"]')
    for c in price_tags1:
        price1.append(c.text.replace('₹','').replace('\n',''))
        
    #scraping product url    
    url_tags1=driver1.find_elements_by_xpath('//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
    for d in url_tags1:
        product_url1.append(d.get_attribute('href'))
    
    next_button=driver1.find_element_by_xpath('//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
    next_button.click()
    time.sleep(5)

In [233]:
driver1.close()

In [231]:
#scraping exchange/return
exchange1=[]
expected_delivery1=[]
availibility1=[]

for i in product_url1:
    driver=webdriver.Chrome('chromedriver.exe')
    driver.get(i)
    time.sleep(5)
    
    #scraping exchange/return
    try:
        exchange_tags1=driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[5]/div[1]/div[1]/div[2]/div[2]/div/div/div[1]/div[15]/div[2]/div/div/div/div[1]/span/div[2]/a')
        exchange1.append(exchange_tags1.text) 
    except NoSuchElementException:
        exchange1.append('-')
    
    #scraping expected delivery
    try:
        expected_delivery1_tags1=driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[5]/div[1]/div[1]/div[2]/div[2]/div/div/div[2]/div[3]/div/div/div/div/form/div/div/div/div/div[3]/div/div[2]/div[8]/div[1]/div/div/div[1]/b')
        expected_delivery1.append(expected_delivery1_tags1.text)
    except NoSuchElementException:
        expected_delivery1.append('-')
        
    #scraping availability    
    try:
        availibility_tags1=driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[5]/div[1]/div[1]/div[2]/div[2]/div/div/div[2]/div[3]/div/div/div/div/form/div/div/div/div/div[3]/div/div[4]/div/div[1]/span')
        availibility1.append(availibility_tags1.text)
    except NoSuchElementException:
        availibility1.append('-')
        
    driver.close()

In [232]:
len(product_url1), len(brand_name1), len(name_of_product1), len(price1), len(exchange1),len(expected_delivery1), len(availibility1)

(181, 181, 181, 181, 181, 181, 181)

In [234]:
#making dataframe
data2={'Brand Name':brand_name1,'Name of the Product':name_of_product1,'Price(₹)':price1,'Return/Exchange':exchange1,'Expected Delivery':expected_delivery1,'Product URL':product_url1,'Availibility':availibility1}
amazon_df=pd.DataFrame(data2)
amazon_df

,Brand Name,Name of the Product,Price(₹),Return/Exchange,Expected Delivery,Product URL,Availibility
0,Satya Paul,Blue Tie for Men,"2,395",30 Days Returns & Exchange,June 6 - 7,https://www.amazon.in/gp/slredirect/picassoRed...,Only 1 left in stock.
1,Satya Paul,Silk WineBurgundy Men Tie,"2,396",30 Days Returns & Exchange,June 6 - 7,https://www.amazon.in/gp/slredirect/picassoRed...,Only 3 left in stock.
2,Satya Paul,Purple Silk Tie for Men,"2,395",30 Days Returns & Exchange,June 6 - 7,https://www.amazon.in/gp/slredirect/picassoRed...,Only 1 left in stock.
3,Satya Paul,Silk Blue Tie for Men,"2,395",30 Days Returns & Exchange,June 6 - 7,https://www.amazon.in/gp/slredirect/picassoRed...,Only 1 left in stock.
4,AXLON,Men's Micro Polyester Necktie Set with Pocket ...,749,10 Days Returns & Exchange,"Sunday, June 5",https://www.amazon.in/Axlon-Polyester-Necktie-...,In stock.
...,...,...,...,...,...,...,...
176,PELUCHE STYLE EVERMORE,Peluche Classy Crimson Red Colored Necktie for...,499,-,"Wednesday, June 1",https://www.amazon.in/Peluche-Classy-Colored-M...,In stock.
177,PELUCHE STYLE EVERMORE,Peluche Classy Crimson Red Colored Necktie for...,499,-,"Wednesday, June 1",https://www.amazon.in/gp/slredirect/picassoRed...,In stock.
178,PELUCHE,Refined Check Black Colored Microfiber Necktie...,499,-,"Thursday, June 2",https://www.amazon.in/gp/slredirect/picassoRed...,In stock.
179,PELUCHE STYLE EVERMORE,"Peluche Elegant Orange, Coral & Peach Colored ...",499,-,"Wednesday, June 1",https://www.amazon.in/gp/slredirect/picassoRed...,In stock.


I have given branded tie as input

In [103]:
#saving as csv file
amazon_df.to_csv('amazon_df.csv')

=======================================================================================================================

Q3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [94]:
# first connect to the web driver
driver3=webdriver.Chrome('chromedriver.exe')
time.sleep(3)

In [95]:
# opening up images.google.com website on automated webdriver
driver3.get('https://www.google.co.in/imghp?hl=en&tab=ri&ogbl')
time.sleep(5)

In [96]:
#skip signin
try:
    skip_signin=driver3.find_element_by_xpath('/html/body/div/c-wiz/div/div/c-wiz/div/div/div/div[2]/div[2]/button')
    skip_signin.click()
except NoSuchElementException:
    print('Exception raised')

Exception raised


In [97]:
#finding elements for user input
search_image1=driver3.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
search_image1.send_keys('fruits')

In [98]:
# clicking search button
click_search_button1=driver3.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button/div')
click_search_button1.click()

In [99]:
for _ in range(20):
    driver3.execute_script('window.scrollBy(0,1000)')
    
fruit_url=[]
fruit = driver3.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

for image  in fruit:
    source=image.get_attribute('src')
    fruit_url.append(source)
    
fruit_urls=fruit_url[-1:-381:-1]

for i in range(len(fruit_urls)):
    if i > 10:
        break
    print("Downloading {0} of {1} images".format(i, 10))
    response=requests.get(fruit_urls[i])
    fruit_file=open(r'C:\Users\Ismail\Desktop\selenium images\fruits'+str(i)+'.jpg','wb')
    fruit_file.write(response.content)

In [100]:
driver3.close()

In [116]:
# first connect to the web driver
driver3a=webdriver.Chrome('chromedriver.exe')
time.sleep(3)

In [117]:
# opening up images.google.com website on automated webdriver
driver3a.get('https://www.google.co.in/imghp?hl=en&tab=ri&ogbl')
time.sleep(5)

In [118]:
#finding elements for user input
search_image3a=driver3a.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
search_image3a.send_keys('cars')

In [119]:
# clicking search button
click_search_button3a=driver3a.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button/div')
click_search_button3a.click()

In [120]:
for _ in range(20):
    driver3a.execute_script('window.scrollBy(0,1000)')
    
car_url=[]
car = driver3a.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

for image  in car:
    source=image.get_attribute('src')
    car_url.append(source)
    
car_urls=car_url[-1:-381:-1]

for i in range(len(car_urls)):
    if i > 10:
        break
    print("Downloading {0} of {1} images".format(i, 10))
    response=requests.get(car_urls[i])
    car_file=open(r'C:\Users\Ismail\Desktop\cars\cars_images'+str(i)+'.jpg','wb')
    car_file.write(response.content)
time.sleep(3)

In [121]:
driver3a.close()

In [122]:
# first connect to the web driver
driver3b=webdriver.Chrome('chromedriver.exe')
time.sleep(3)

In [123]:
# opening up images.google.com website on automated webdriver
driver3b.get('https://www.google.co.in/imghp?hl=en&tab=ri&ogbl')
time.sleep(5)

In [125]:
#finding elements for user input
search_image3b=driver3b.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
search_image3b.send_keys('Machine learning')

In [126]:
# clicking search button
click_search_button3b=driver3b.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button/div')
click_search_button3b.click()

In [127]:
for _ in range(20):
    driver3b.execute_script('window.scrollBy(0,1000)')
    
machine_learning_url=[]
machine_learning = driver3b.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

for image  in machine_learning:
    source=image.get_attribute('src')
    machine_learning_url.append(source)
    
machine_learning_urls=machine_learning_url[-1:-381:-1]

for i in range(len(machine_learning_urls)):
    if i > 10:
        break
    print("Downloading {0} of {1} images".format(i, 10))
    response=requests.get(machine_learning_urls[i])
    machine_learning_file=open(r'C:\Users\Ismail\Desktop\machine_learning\machine_learning_images'+str(i)+'.jpg','wb')
    machine_learning_file.write(response.content)
time.sleep(3)

In [128]:
driver3b.close()

In [129]:
# first connect to the web driver
driver3c=webdriver.Chrome('chromedriver.exe')
time.sleep(3)

In [130]:
# opening up images.google.com website on automated webdriver
driver3c.get('https://www.google.co.in/imghp?hl=en&tab=ri&ogbl')
time.sleep(5)

In [131]:
#finding elements for user input
search_image3c=driver3c.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
search_image3c.send_keys('guitar')

In [132]:
# clicking search button
click_search_button3c=driver3c.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button/div')
click_search_button3c.click()

In [133]:
for _ in range(20):
    driver3c.execute_script('window.scrollBy(0,1000)')
    
guitar_url=[]
guitar = driver3c.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

for image  in guitar:
    source=image.get_attribute('src')
    guitar_url.append(source)
    
guitar_urls=guitar_url[-1:-381:-1]

for i in range(len(guitar_urls)):
    if i > 10:
        break
    print("Downloading {0} of {1} images".format(i, 10))
    response=requests.get(guitar_urls[i])
    guitar_file=open(r'C:\Users\Ismail\Desktop\guitar\guitar_images'+str(i)+'.jpg','wb')
    guitar_file.write(response.content)
time.sleep(3)

In [134]:
driver3c.close()

In [135]:
# first connect to the web driver
driver3d=webdriver.Chrome('chromedriver.exe')
time.sleep(3)

In [136]:
# opening up images.google.com website on automated webdriver
driver3d.get('https://www.google.co.in/imghp?hl=en&tab=ri&ogbl')
time.sleep(5)

In [137]:
#finding elements for user input
search_image3d=driver3d.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
search_image3d.send_keys('cakes')

In [138]:
# clicking search button
click_search_button3d=driver3d.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button/div')
click_search_button3d.click()

In [139]:
for _ in range(20):
    driver3d.execute_script('window.scrollBy(0,1000)')
    
cake_url=[]
cake = driver3d.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

for image  in cake:
    source=image.get_attribute('src')
    cake_url.append(source)
    
cake_urls=cake_url[-1:-381:-1]

for i in range(len(cake_urls)):
    if i > 10:
        break
    print("Downloading {0} of {1} images".format(i, 10))
    response=requests.get(cake_urls[i])
    cake_file=open(r'C:\Users\Ismail\Desktop\selenium images\cake_image'+str(i)+'.jpg','wb')
    cake_file.write(response.content)
time.sleep(3)

In [140]:
driver3d.close()

=======================================================================================================================

Q4.Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [86]:
# first connect to the web driver
driver4=webdriver.Chrome('chromedriver.exe')
time.sleep(5)

In [87]:
# opening up flipkart.com website on automated webdriver
driver4.get('https://www.flipkart.com/')

In [88]:
skip_login4=driver4.find_element_by_xpath('/html/body/div[2]/div/div/button')
skip_login4.click()

In [50]:
#finding elements for user input
search_product4=driver4.find_element_by_xpath('/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')
search_product4.send_keys('Oneplus Nord, pixel 4A')

In [51]:
# clicking search button
search_button4=driver4.find_element_by_xpath('/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button')
search_button4.click()

In [14]:
#product_url
product_url=[]
url_tags4=driver4.find_elements_by_xpath('//a[@class="_1fQZEK"]')
for d in url_tags4:
    product_url.append(d.get_attribute('href'))

In [62]:
#scraping product_details
brand_name4=[]
smartphone_name4=[]
colour4=[]
RAM4=[]
storage4=[]
primary_camera4=[]
secondary_camera4=[]
display_size4=[]
battery_capacity4=[]
price4=[]
for i in product_url:
    driver=webdriver.Chrome('chromedriver.exe')
    driver.get(i)
    time.sleep(5)
    
    read_more4=driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')
    read_more4.click()
    
#scraping brand name
    try:
        brand_tags4=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span')
        brand_name4.append(brand_tags4.text)
    except NoSuchElementException:
        brand_name4.append('-')
        
#scraping smart phone name
    try:
        smartphone_name4_tags4=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[1]/table/tbody/tr[3]/td[2]/ul/li')
        smartphone_name4.append(smartphone_name4_tags4.text)
    except NoSuchElementException:
        smartphone_name4.append('-')
        
#scraping colour    
    try:
        colour4_tags4=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[1]/table/tbody/tr[4]/td[2]/ul/li')
        colour4.append(colour4_tags4.text)
    except NoSuchElementException:
        colour4.append('-')
        
#scraping RAM        
    try:
        RAM4_tags4=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[4]/table/tbody/tr[2]/td[2]/ul/li')
        RAM4.append(RAM4_tags4.text)
    except NoSuchElementException:
        RAM4.append('-')
        
#scraping storage        
    try:
        storage_tags4=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[4]/table/tbody/tr[1]/td[2]/ul/li')
        storage4.append(storage_tags4.text)
    except NoSuchElementException:
        storage4.append('-')
        
#scraping primary camera details        
    try:
        primary_camera4_tags4=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[5]/table/tbody/tr[1]/td[2]/ul/li')
        primary_camera4.append(primary_camera4_tags4.text)
    except NoSuchElementException:
        primary_camera4.append('-')
        
#scraping secondary camera details        
    try:
        secondary_camera4_tags4=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[7]/div[1]/div/div[2]/ul/li[3]')
        secondary_camera4.append(secondary_camera4_tags4.text)
    except NoSuchElementException:
        secondary_camera4.append('-')
        
#scraping display details        
    try:
        display_size4_tags4=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[2]/table/tbody/tr[1]/td[2]/ul/li')
        display_size4.append(display_size4_tags4.text)
    except NoSuchElementException:
        display_size4.append('-')
        
#scraping battery details    
    try:
        battery_capacity4_tags4=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[7]/table/tbody/tr/td[2]/ul/li')
        battery_capacity4.append(battery_capacity4_tags4.text)
    except NoSuchElementException:
        battery_capacity4.append('-')
        
#scraping price    
    try:
        price4_tags4=driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')
        price4.append(price4_tags4.text.replace('₹',''))
    except NoSuchElementException:
        price4.append('-')
        
#closing driver        
    driver.close()

In [74]:
len(price4),len(brand_name4),len(colour4),len(RAM4),len(storage4),len(primary_camera4), len(secondary_camera4),len(display_size4), len(battery_capacity4),    len(smartphone_name4)

(24, 24, 24, 24, 24, 24, 24, 24, 24, 24)

In [89]:
driver4.close()

In [83]:
#making dataframe
data4={'Brand_Name':brand_name4 ,'Smartphone_name':smartphone_name4,'Colour':colour4,'RAM':RAM4,'Storage(ROM)':storage4,'Primary_Camera':primary_camera4,'secondary_camera':secondary_camera4, 'Display_Size':display_size4, 'Battery_Capacity':battery_capacity4, 'Price(₹)':price4, 'Product_URL':product_url}
smart_phone_df=pd.DataFrame(data4)

#dataframe
smart_phone_df

,Brand_Name,Smartphone_name,Colour,RAM,Storage(ROM),Primary_Camera,secondary_camera,Display_Size,Battery_Capacity,Price(₹),Product_URL
0,"OnePlus Nord 2 5G (Blue Haze, 128 GB) (8 GB RAM)",Nord 2 5G,Blue Haze,8 GB,128 GB,50MP Rear Camera,50MP Rear Camera,16.33 cm (6.43 inch),4500 mAh,"29,735",https://www.flipkart.com/oneplus-nord-2-5g-blu...
1,"OnePlus Nord 2 5G (Gray Sierra, 128 GB) (8 GB...",Nord 2 5G,Gray Sierra,8 GB,128 GB,50MP Rear Camera,50MP Rear Camera,16.33 cm (6.43 inch),4500 mAh,"28,753",https://www.flipkart.com/oneplus-nord-2-5g-gra...
2,"OnePlus 9RT 5G (Nano Silver, 256 GB) (12 GB RAM)",-,-,-,-,-,50MP Rear Camera,-,-,"44,999",https://www.flipkart.com/oneplus-9rt-5g-nano-s...
3,"OnePlus 9 5G (Arctic Sky, 128 GB) (8 GB RAM)",9 5G,Arctic Sky,8 GB,128 GB,48MP Rear Camera,48MP Rear Camera,16.64 cm (6.55 inch),4500 mAh,"40,999",https://www.flipkart.com/oneplus-9-5g-arctic-s...
4,"OnePlus 9 Pro 5G (Morning Mist, 128 GB) (8 GB...",9 Pro 5G,Morning Mist,8 GB,128 GB,48MP Rear Camera,48MP Rear Camera,17.02 cm (6.7 inch),4500 mAh,"49,678",https://www.flipkart.com/oneplus-9-pro-5g-morn...
5,"OnePlus 9 Pro 5G (Morning Mist, 256 GB) (12 G...",9 Pro 5G,Morning Mist,12 GB,256 GB,48MP Rear Camera,48MP Rear Camera,17.02 cm (6.7 inch),4500 mAh,"54,990",https://www.flipkart.com/oneplus-9-pro-5g-morn...
6,"OnePlus 10 Pro 5G (Volcanic Black, 256 GB) (1...",-,-,-,-,-,48MP Rear Camera,-,-,"71,998",https://www.flipkart.com/oneplus-10-pro-5g-vol...
7,"OnePlus 8T 5G (Aquamarine Green, 128 GB) (8 G...",-,-,-,-,-,-,-,-,"39,890",https://www.flipkart.com/oneplus-8t-5g-aquamar...
8,"OnePlus 8T 5G (Lunar Silver, 128 GB) (8 GB RAM)",-,-,-,-,-,-,-,-,"41,999",https://www.flipkart.com/oneplus-8t-5g-lunar-s...
9,"OnePlus 9RT 5G (Hacker Black, 256 GB) (12 GB ...",9RT 5G,Hacker Black,12 GB,256 GB,50MP Rear Camera,50MP Rear Camera,16.81 cm (6.62 inch),4500 mAh,"43,999",https://www.flipkart.com/oneplus-9rt-5g-hacker...


In [84]:
#saving as csv file
smart_phone_df.to_csv('smart_phone_df.csv')

======================================================================================================================

Q5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [90]:
# first connect to the web driver
driver5=webdriver.Chrome('chromedriver.exe')
time.sleep(5)

In [91]:
# opening up google maps website on automated webdriver
driver5.get('https://www.google.co.in/maps/@12.9545163,77.3500502,10z')

In [93]:
#finding elements for location
search_location5=driver5.find_element_by_xpath('/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/form/div[2]/div[3]/div/input[1]')
search_location5.send_keys('bangalore')

In [95]:
search_button5=driver5.find_element_by_xpath('/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/button')
search_button5.click()

In [96]:
#exctracting url
try:
    location_url=driver5.current_url
    print('Location_URL:',location_url)
    
except NoSuchElementException:
    print('not extractable')

Location_URL: https://www.google.co.in/maps/place/Bengaluru,+Karnataka/@12.9539974,77.6309395,10z/data=!4m5!3m4!1s0x3bae1670c9b44e6d:0xf8dfc3e8517e4fe0!8m2!3d12.9715987!4d77.5945627


In [41]:
#importing regex module
import re

In [132]:
sort_numbers=r'[0-9]+'   #sorting only numbers from the extracted link
x=re.findall(sort_numbers, location_url) #finding numbers and keeping it in list 'x'
x1=x[0]
x2=x[1]
x3=x[2]
x4=x[3]
latitude=x1+str('.')+x2 #assigning latitude 
longitude=x3+str('.')+x4 #assigning longitude

In [133]:
#prining latitude
latitude 

'12.9539974'

In [134]:
#printing longitude
longitude

'77.6309395'

In [135]:
#closing driver5 window
driver5.close()

=====================================================================================================================

Q6.Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [2]:
# first connect to the web driver
driver6=webdriver.Chrome('chromedriver.exe')
time.sleep(5)

In [3]:
# opening up trak.in website on automated webdriver
driver6.get('https://trak.in/')

In [4]:
#clicking on funding deals
click_funding_deals=driver6.find_element_by_xpath('/html/body/div[1]/header/div[2]/div/div/div/div/nav/ul/li[9]/a')
click_funding_deals.click()

In [54]:
#scraping details column wise
column1=[]
column_tags1=driver6.find_elements_by_xpath('//td[@class="column-1"]')
for i in column_tags1:
    column1.append(i.text)

march_c1=column1[5:12]
feb_c1=column1[12:21]
jan_c1=column1[21:29]

In [56]:
column2=[]
column_tags2=driver6.find_elements_by_xpath('//td[@class="column-2"]')
for i in column_tags2:
    column2.append(i.text)

march_c2=column2[5:12]
feb_c2=column2[12:21]
jan_c2=column2[21:29]

In [57]:
column3=[]
column_tags3=driver6.find_elements_by_xpath('//td[@class="column-3"]')
for i in column_tags3:
    column3.append(i.text)

march_c3=column3[5:12]
feb_c3=column3[12:21]
jan_c3=column3[21:29]

In [59]:
column4=[]
column_tags4=driver6.find_elements_by_xpath('//td[@class="column-4"]')
for i in column_tags4:
    column4.append(i.text)

march_c4=column4[5:12]
feb_c4=column4[12:21]
jan_c4=column4[21:29]

In [61]:
column5=[]
column_tags5=driver6.find_elements_by_xpath('//td[@class="column-5"]')
for i in column_tags5:
    column5.append(i.text)

march_c5=column5[5:12]
feb_c5=column5[12:21]
jan_c5=column5[21:29]

In [63]:
column6=[]
column_tags6=driver6.find_elements_by_xpath('//td[@class="column-6"]')
for i in column_tags6:
    column6.append(i.text)

march_c6=column6[5:12]
feb_c6=column6[12:21]
jan_c6=column6[21:29]

In [64]:
column7=[]
column_tags7=driver6.find_elements_by_xpath('//td[@class="column-7"]')
for i in column_tags7:
    column7.append(i.text)

march_c7=column7[5:12]
feb_c7=column7[12:21]
jan_c7=column7[21:29]

In [65]:
column8=[]
column_tags8=driver6.find_elements_by_xpath('//td[@class="column-8"]')
for i in column_tags8:
    column8.append(i.text)

march_c8=column8[5:12]
feb_c8=column8[12:21]
jan_c8=column8[21:29]

In [73]:
column9=[]
column_tags9=driver6.find_elements_by_xpath('//td[@class="column-9"]')
for i in column_tags9:
    column9.append(i.text)

march_c9=column9[5:12]
feb_c9=column9[12:21]
jan_c9=column9[21:29]

In [74]:
col=['Sr.No.','Date(dd/mm/yyyy)','Startup_Name','Industry/Vertical','Sub-Vertical','City/Location',"Investors'_Name",'Investment_Type','Amount(In USD)']

In [78]:
driver6.close()

In [75]:
#making dataframe for jan 2021

jan_2021_df=pd.DataFrame(list(zip(jan_c1,jan_c2,jan_c3,jan_c4,jan_c5,jan_c6,jan_c7,jan_c8,jan_c9)))
jan_2021_df.columns=col

#dataframe
jan_2021_df

,Sr.No.,Date(dd/mm/yyyy),Startup_Name,Industry/Vertical,Sub-Vertical,City/Location,Investors'_Name,Investment_Type,Amount(In USD)
0,1,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,2,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,3,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,4,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,5,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,6,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,7,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,8,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"


In [76]:
#making dataframe for feb 2021

feb_2021_df=pd.DataFrame(list(zip(feb_c1,feb_c2,feb_c3,feb_c4,feb_c5,feb_c6,feb_c7,feb_c8,feb_c9)))
feb_2021_df.columns=col

#dataframe
feb_2021_df

,Sr.No.,Date(dd/mm/yyyy),Startup_Name,Industry/Vertical,Sub-Vertical,City/Location,Investors'_Name,Investment_Type,Amount(In USD)
0,1,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
1,2,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"
2,3,19/02/2021,Fingerlix,Hospitality,Semi-cooked food delivery app,Mumbai,"Rhodium Trust, Accel Partners and Swiggy",Series C,"2,747,045.20"
3,4,17/02/2021,Zolve,FinTech,Global Neobank Venture,Mumbai,Accel Partners and Lightspeed Venture Partners,Seed,"1,50,00,000"
4,5,15/02/2021,KreditBee,Finance,Digital lending platform,Bengaluru,Azim Premji’s PremjiInvest and South Korea’s M...,Series C,"75,000,000"
5,6,12/02/2021,Pepperfry,E-commerce,Multi-brand furniture brand,Mumbai,InnoVen Capital,Debt Financing,"4,773,958"
6,7,12/02/2021,Grofers,E-Commerce,Online supermarket,Gurgaon,SoftBank Vision Fund (SVF),Unspecified,"55,000,000"
7,8,09/02/2021,Nothing,Technology,Consumer Technology Venture,London,GV,Series A,"15,000,000"
8,9,09/02/2021,SplashLearn,EdTech,Game-based learning programme,Gurgaon,Owl Ventures,Series C,"18,000,000"


In [77]:
#making dataframe for march 2021

march_2021_df=pd.DataFrame(list(zip(march_c1,march_c2,march_c3,march_c4,march_c5,march_c6,march_c7,march_c8,march_c9)))
march_2021_df.columns=col

#dataframe
march_2021_df

,Sr.No.,Date(dd/mm/yyyy),Startup_Name,Industry/Vertical,Sub-Vertical,City/Location,Investors'_Name,Investment_Type,Amount(In USD)
0,1,04/03/2021,DealShare,E-commerce,Online shopping platform,"Jaipur, Rajasthan",Innoven Capital,Debt Financing,"250,000,000"
1,2,31/03/2021,Uniphore,Technology,Conversational Service Automation (CSA),Palo Alto,Sorenson Capital Partners,Series D,"140,000,000"
2,3,30/03/2021,Dunzo,E-commerce,Hyper-local delivery app,Bengaluru,Krishtal Advisors Pte Ltd,Series E,"8,000,000"
3,4,30/03/2021,BYJU’S,Edu-tech,Online tutoring,Bengaluru,"MC Global Edtech, B Capital, Baron, others",Series F,"460,000,000"
4,5,23/03/2021,SkilloVilla,Edu-tech,Career and job-oriented upskilling.,Bengaluru,"Titan Capital, others",Seed,"300,000,000"
5,6,25/03/2021,CityMall,E-commerce,Social ecommerce and online grocery platform,Gurgaon,Accel Partners,Series A,"11,000,000"
6,7,26/03/2021,DotPe,FinTech,Commerce and payments platform to offline ente...,Gurgaon,PayU,Series A,"27,500,000"


=======================================================================================================================

7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [187]:
# first connect to the web driver
driver7=webdriver.Chrome('chromedriver.exe')
time.sleep(5)

In [188]:
# opening up digit.in website on automated webdriver
driver7.get('https://www.digit.in/')

In [189]:
#clicking on best gaming laptops
click_laptop=driver7.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[2]/div[4]/ul/li[9]/a')
click_laptop.click()

In [190]:
#scraping details of laptops
name7=[]
OS7=[]
display7=[]
processor7=[]
memory7=[]
price7=[]
EMI7=[]

#scraping name
name7b=[]
name_tags7=driver7.find_elements_by_xpath('//div[@class="Top10-Seller"]//td')
for i in name_tags7:
    name7b.append(i.text.replace('₹',''))
name7=name7b[0::3]

#price
price7=name7b[2::3]

#scraping EMI
emi7b=[]
emi_tags7=driver7.find_elements_by_xpath('//table[@class="table"]/tbody/tr/td/strong')
for p in emi_tags7:
    emi7b.append(p.text.replace('₹',''))
EMI7=emi7b[1:8:2]
EMI7.append(emi7b[10])
EMI7.append(emi7b[12])
EMI7.append(emi7b[15])
EMI7.append(emi7b[18])
EMI7.append(emi7b[20])
EMI7.append(emi7b[22])

#scarping specifications
specification7b=[]
specification_tags7=driver7.find_elements_by_xpath('//div[@class="value"]')
for p in specification_tags7:
    specification7b.append(p.text.replace('OS','').replace('DISPLAY','').replace('PROCESSOR','').replace('MEMORY',''))
    
#OS
OS7=specification7b[0::4]

#display
display7=specification7b[1::4]

#processor
processor7=specification7b[2::4]

#memory
memory7=specification7b[3::4]

In [191]:
#checking lenght
len(name7), len(OS7), len(display7), len(processor7), len(memory7), len(price7), len(EMI7)

(10, 10, 10, 10, 10, 10, 10)

In [192]:
driver7.close()

In [193]:
#making dataframe
gaming_laptops_df=pd.DataFrame(list(zip(name7,OS7,display7,processor7,memory7,price7,EMI7)))
gaming_laptops_df.columns=['Name','OS','Display','Processor','Memory','Price(₹)','EMI']

#dataframe
gaming_laptops_df

,Name,OS,Display,Processor,Memory,Price(₹),EMI
0,MSI Raider GE76,WINDOWS 11 HOME,"17"" (3840 X 2160)",12TH GEN INTEL CORE I9-12900HK | 5 GHZ,2 TB SSD/16 GBGB DDR5,"429,940","71,656/MONTH"
1,ASUS ROG Strix Scar 15,WINDOWS 11 HOME,"15.6"" (2560 X 1440)",12TH GEN INTEL CORE I9-12900H | 2.5 GHZ,2 TB SSD/32 GBGB DDR5,"285,390","47,565/MONTH"
2,Acer Nitro 5,WINDOWS 10,"15.6"" (1920 X 1080)",AMD RYZEN 9 OCTA CORE | 2.4 GHZ,1 TB HDD/16 GBGB DDR4,"129,990","21,665/MONTH"
3,MSI Stealth 15M,WINDOWS 10,"15.6"" (1920 X 1080)",INTEL CORE I7 11TH GEN - 11375H | NA,1 TB SSD/16 GBGB DDR4,"134,990","22,498/MONTH"
4,ASUS ROG Strix Scar 15,WINDOWS 10,"15.6"" (2560 X 1440)",AMD RYZEN 9 OCTA CORE - 5900HX | 3.3 GHZ,2 TB SSD/32 GBGB DDR4,"193,990","32,331/MONTH"
5,ASUS ROG Strix Scar 15,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",AMD RYZEN™ 9 5900HX | 3.3 GHZ,1 TB SSD/16 GBGB DDR4,"215,990","35,998/MONTH"
6,ASUS ZEPHYRUS G14,WINDOWS 10 HOME,"14"" (1920 X 1080)",AMD 3RD GEN RYZEN 9 | 3.3 GHZ,1 TB SSD/16 GBGB DDR4,"144,990","24,165/MONTH"
7,HP Omen 16,WINDOWS 11 HOME,"16.1"" (1920 X 1080)",12TH GEN INTEL CORE I7-12700H | 4.7 GHZ,1 TB SSD/16 GBGB DDR4,"139,990","23,331/MONTH"
8,ASUS ROG ZEPHYRUS DUO 15,WINDOWS 10,"15.6"" (3840 X 1100)",INTEL CORE I7 10TH GEN 10875H | NA,512 GB SSD/4 GBGB DDR4,"185,000","30,833/MONTH"
9,Acer Aspire 7 gaming laptop,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",AMD RYZEN™ 5-5500U HEXA-CORE | NA,512 GB SSD/8 GBGB DDR4,"53,490","8,915/MONTH"


======================================================================================================================

8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [2]:
# first connect to the web driver
driver8=webdriver.Chrome('chromedriver.exe')
time.sleep(5)

In [3]:
# opening up forbes.com website on automated webdriver
driver8.get('https://www.forbes.com/?sh=5d9deba82254') 

In [4]:
#click explore
click_explore8=driver8.find_element_by_xpath('/html/body/div[1]/header/nav/div[1]/button[1]')
click_explore8.click()

In [7]:
#click_billinaire
click_billinaire8=driver8.find_element_by_xpath('/html/body/div[1]/header/nav/div[3]/ul/li[1]')
click_billinaire8.click()

In [8]:
#click world billinaire
world_billinaire8=driver8.find_element_by_xpath('/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a')
world_billinaire8.click()

In [32]:
#scraping name
name8=[]
name_tags8=driver8.find_elements_by_xpath('//div[@class="personName"]/div')
for i in name_tags8:
    name8.append(i.text)
    
#scraping rank
rank8=[]
rank_tags8=driver8.find_elements_by_xpath('//div[@class="rank"]')
for i in rank_tags8:
    rank8.append(i.text)
    
#scraping networth
networth8=[]
networth_tags8=driver8.find_elements_by_xpath('//div[@class="netWorth"]/div')
for i in networth_tags8:
    networth8.append(i.text)
    
#scraping age
age8=[]
age_tags8=driver8.find_elements_by_xpath('//div[@class="age"]/div')
for i in age_tags8:
    age8.append(i.text)
    
#scraping citizenship
citizenship8=[]
citizenship_tags8=driver8.find_elements_by_xpath('//div[@class="countryOfCitizenship"]')
for i in citizenship_tags8:
    citizenship8.append(i.text)
    
#scraping source
source8=[]
source_tags8=driver8.find_elements_by_xpath('//span[@class="source-text"]')
for i in source_tags8:
    source8.append(i.text)
    
#scraping industry
industry8=[]
industry_tags8=driver8.find_elements_by_xpath('//div[@class="category"]/div')
for i in industry_tags8:
    industry8.append(i.text)

In [33]:
#checking length
len(name8), len(rank8), len(networth8), len(age8), len(citizenship8), len(source8), len(industry8)

(200, 200, 200, 200, 200, 200, 200)

In [35]:
driver8.close()

In [34]:
#making dataframe
data8={'Rank':rank8,'Name':name8,'Net_Worth':networth8,'Age':age8,'Citizenship':citizenship8,'Source':source8,'Industry':industry8}
world_billinaires_df=pd.DataFrame(data8)

#dataframe
world_billinaires_df

,Rank,Name,Net_Worth,Age,Citizenship,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,72,Brazil,beer,Food & Beverage
196,197.,Leon Black,$10 B,70,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,$10 B,40,United States,Airbnb,Technology
198,197.,David Geffen,$10 B,79,United States,"movies, record labels",Media & Entertainment


=======================================================================================================================

Q9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [74]:
# first connect to the web driver
driver9=webdriver.Chrome('chromedriver.exe')
time.sleep(5)

In [75]:
# opening up youtube video website on automated webdriver
driver9.get('https://youtu.be/rdi7G1hY4-w')

In [77]:
#scrolling down
for _ in range(100):
    driver9.execute_script('window.scrollBy(0,1000)')
    

In [70]:
#scraping details 
comments9=[]
comment_upvotes9=[]
times9=[]

#scraping comments
comment_tags9=driver9.find_elements_by_id("content-text")
for i in comment_tags9:
    comments9.append(i.text)

In [87]:
#scraping time
time_tags9=driver9.find_elements_by_xpath('//a[@class="yt-simple-endpoint style-scope yt-formatted-string"]')
for i in time_tags9:
    time9.append(i.text.replace('(edited)',''))

In [83]:
#scrapping comment upvotes
comment_upvote_tags9=driver9.find_elements_by_id('vote-count-middle')
for i in comment_upvote_tags9:
    comment_upvotes9.append(i.text)

In [117]:
comment9=comments9[0:502]
comment_upvote9=comment_upvotes9[0:502]
times9=time9[64:566]

In [118]:
len(comment9), len(comment_upvote9), len(times9)

(502, 502, 502)

In [119]:
driver9.close()

In [120]:
#making dataframe
data9={'Comments':comment9,'Comment_upvotes':comment_upvote9,'Time_when_comment_posted':times9}
youtube_video_df9=pd.DataFrame(data9)

#dataframe
youtube_video_df9

,Comments,Comment_upvotes,Time_when_comment_posted
0,This song is filled with all the feelings,16K,9 months ago
1,"Don't worry my dear brother,I wish your countr...",373,2 weeks ago
2,I thought it was a meme but when I listened to...,1.2K,1 month ago
3,“My heart is like an open highway” 😢\n\nLove f...,279,2 weeks ago
4,2021 : Great music\n2022 : this music never ge...,508,1 month ago
...,...,...,...
497,GOOD JOB BUDDY!,,2 weeks ago
498,He proved that confidence is more important th...,233,2 weeks ago
499,His starring alone give joy ❤🥰,1,1 month ago
500,It's my life \nIt's now or never \nAnd autotun...,273,10 days ago


=======================================================================================================================

Q10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [121]:
# first connect to the web driver
driver10=webdriver.Chrome('chromedriver.exe')
time.sleep(5)

In [122]:
# opening up hostelworld website on automated webdriver
driver10.get('https://www.hostelworld.com/')

In [126]:
#search_location
search_location10=driver10.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div/div/input')
search_location10.send_keys('London')

In [128]:
#click_lets_go
click_letsgo=driver10.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[5]/button')
click_letsgo.click()

In [131]:
hostel_name10=[]
#scarping hostel name
name_tags10=driver10.find_elements_by_xpath('//h2[@class="title title-6"]')
for i in name_tags10:
    hostel_name10.append(i.text)

In [141]:
#scraping distance from city
distance10=[]
distance_tags10=driver10.find_elements_by_xpath('//span[@class="description"]')
for i in distance_tags10:
    distance10.append(i.text.replace('Hostel - ','').replace('Bed and Breakfast - ','').replace(' from city centre','').replace('Hotel - ',''))

In [162]:
ratings10=[]
#scraping ratings
    
rating_tags10=driver10.find_elements_by_xpath('//div[@class="score orange big"]')
for i in rating_tags10:
    ratings10.append(i.text)

In [164]:
total_reviews10=[]
#scraping total reviews
review_tags10=driver10.find_elements_by_xpath('//div[@class="reviews"]')
for i in review_tags10:
    total_reviews10.append(i.text.replace('Total Reviews',''))

In [178]:
#scraping private from price
privates_from_price10=[]
private_p10=driver10.find_elements_by_xpath('//div[@class="price title-5"]')
for i in private_p10:
    privates_from_price10.append(i.text.replace('Rs','').replace('30031.49 ','').replace('12279.13 ','').replace('29119.23 ','').replace('21070.27 ',''))
    
privates_from_price10.remove('7916.8 7521')

In [182]:
#scraping dorms from price
dorms_from_price10=['-','-','-','7916.8 7521','-','-','-','-','-','-','-','-','-','-','-','-']

In [185]:
product_url10=[]
product_url_tags10=driver10.find_elements_by_xpath('//div[@class="gallery"]/a')
for i in product_url_tags10:
    product_url10.append(i.get_attribute('href'))

In [189]:
 product_url10.pop(0)

'https://www.hostelworld.com/pwa/hosteldetails.php/Selina-Camden/London/310274?from=2022-05-31&to=2022-06-03&guests=2'

In [191]:
product_url10.pop(9)

'https://www.hostelworld.com/pwa/hosteldetails.php/Belgrave-Hotel-London/London/268771?from=2022-05-31&to=2022-06-03&guests=2'

In [193]:
len(product_url10)

16

In [209]:
#scraping facilities
facilities10a=[]
facilities10b=[]
property_description10=[]

for i in product_url10:
    driver=webdriver.Chrome('chromedriver.exe')
    driver.get(i)
    time.sleep(5)
    
#scraping facilities for first 2 hostels    
    try:
        facility_tags10a=driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/section/div[11]/div/ul')
        facilities10a.append(facility_tags10a.text.replace('\n',' '))
    except NoSuchElementException:
        facilities10a.append('-')
        
#scraping facilities from 3rd hostel    
    try:
        facility_tags10=driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/section/div[10]/div')
        facilities10b.append(facility_tags10.text.replace('\n',' '))
    except NoSuchElementException:
        facilities10b.append('-')
        
#scraping property description        
    try:
        des_tags10=driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/section/div[6]/div/div[2]/div/div/div[2]')
        property_description10.append(des_tags10.text.replace('\n',' '))
    except NoSuchElementException:
        property_description10.append('-')
        
    driver.close()

In [221]:
#storing data in facilities list
facilities10=[]
facilities10.append(facilities10a[0])
facilities10.append(facilities10a[1])
facilities10.extend(facilities10b[2::])

In [ ]:
rating10=[]

In [222]:
len(facilities10)

16

In [208]:
len(property_description10)

16

In [239]:
ratings10

['9.1', '7.2', '8.0', '6.6', '8.6', '8.6', '7.1', '7.4', '9.1']

In [240]:
rating10=['9.1','7.2','8.0','6.6','8.6','8.6','7.1','-','7.4','9.1','-','-','-','-','-','-']

In [241]:
len(hostel_name10), len(distance10), len(rating10), len(total_reviews10), len(privates_from_price10), len(dorms_from_price10), len(facilities10), len(property_description10)

(16, 16, 16, 16, 16, 16, 16, 16)

In [242]:
driver10.close()

In [243]:
#making dataframe
data10={'Hostel_name':hostel_name10,'distance_from_city_centre':distance10,'ratings':rating10,'total reviews':total_reviews10,'privates_from_price(Rs)':privates_from_price10,'dorms_from_price(Rs)':dorms_from_price10,'Facilities':facilities10,'Property_description':property_description10}
hostels_df=pd.DataFrame(data10)

#dataframe
hostels_df

,Hostel_name,distance_from_city_centre,ratings,total reviews,privates_from_price(Rs),dorms_from_price(Rs),Facilities,Property_description
0,Selina Camden,5.5km,9.1,25,25527,-,Free Linen Included Towels Included Free WiFi ...,"Among underground music venues, innovative mus..."
1,Safestay London Elephant & Castle,1.7km,7.2,4256,14721,-,Free Linen Included Free City Maps Free WiFi G...,Safestay at Elephant & Castle is ideal if you ...
2,Smart Hyde Park View Hostel,5km,8.0,4550,20242,-,Facilities Free Linen Included Free City Maps ...,Our Hyde Park View Hostel is in a superb locat...
3,Saint James Backpackers,5.5km,6.6,1790,11665,7916.8 7521,Facilities Free Free Breakfast Linen Included ...,
4,Barry House,4.2km,8.6,10,21820,-,Facilities Free Free Breakfast Free City Maps ...,Barry House is a Central London bed and breakf...
5,London House Hotel,5.3km,8.6,1381,20383,-,Facilities Free Free City Maps Free WiFi Gener...,Our lovely boutique hotel is conveniently loca...
6,Elmwood Hotel,3.2km,7.1,115,12104,-,Facilities Free Free Internet Access General H...,This central London hotel just 2-minutes from ...
7,Cranbrook Hotel,14.8km,-,58,8342,-,Facilities Free Free Breakfast Free Parking Ge...,-
8,Colliers Hotel,2.1km,7.4,4,12824,-,Facilities General Key Card Access Adaptors Ha...,Colliers Hotel offers basic budget accommodati...
9,City Hotel Carlton,1.8km,9.1,40,23358,-,Facilities Free Free Internet Access General S...,Carlton hotel is conveniently situated within ...
